# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [17]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [18]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [19]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
# extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 
            
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [20]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

256


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [21]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [22]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [23]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

## Question 1

### Description: Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

### Create table songs_by_session

In [24]:
## Drop table irst if it already exists
try:
    session.execute('DROP TABLE IF EXISTS songs_by_session')
except Exception as e:
    print(e)
                    
## Create table
query = "CREATE TABLE IF NOT EXISTS songs_by_session "
query = query + "(session_id int, item_in_session int, artist text, song_title text, duration float, PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

### Read csv file and populate songs_by_session table

In [25]:
file = 'event_datafile_new.csv'

# read csv and populate table
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songs_by_session (session_id, item_in_session, artist, song_title, duration)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

### Run the query to get the answer for Question 1

In [28]:
query = "SELECT artist, song_title, duration FROM songs_by_session WHERE session_id = 338 AND item_in_session = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
# display results on the screen
for row in rows:
    print (row.artist, row.song_title, row.duration)

## Question 2

### Description: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

### Create table songs_by_session_and_user

In [31]:
## Drop table first if it already exists
try:
    session.execute('DROP TABLE IF EXISTS songs_by_session_and_user')
except Exception as e:
    print(e)
                    
## Create table
query = "CREATE TABLE IF NOT EXISTS songs_by_session_and_user "
query = query + "(user_id int, session_id int, item_in_session int, artist text, song_title text, first_name text,"
query = query + " last_name text, PRIMARY KEY ((user_id, session_id), item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)  

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Read csv and populate table

In [37]:
file = 'event_datafile_new.csv'

# read csv file and populate table
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songs_by_session_and_user (user_id, session_id, item_in_session, artist, song_title, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

### Run the query to get the answer for Question 2

In [ ]:
query = "SELECT artist, song_title, first_name, last_name FROM songs_by_session_and_user WHERE user_id = 10 AND session_id = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
# display results
for row in rows:
    print (row.artist, row.song_title, row.first_name, row.last_name)

## Question 3

### Description: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

### Create table, read csv file, populate table, run query to get answer for question 3

In [39]:
## Drop table first if it already exists
try:
    session.execute('DROP TABLE IF EXISTS user_listen_song')
except Exception as e:
    print(e)
                    
##  Create table
query = "CREATE TABLE IF NOT EXISTS user_listen_song "
query = query + "(song_title text, user_id int, first_name text,"
query = query + " last_name text, PRIMARY KEY (song_title, user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)  
    
file = 'event_datafile_new.csv'

# read file and populate table
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_listen_song (song_title, user_id, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))
    

query = "SELECT first_name, last_name FROM user_listen_song WHERE song_title = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.first_name, row.last_name)
                    

Sara Johnson


### Drop the tables before closing out the sessions

In [13]:
query = "DROP TABLE IF EXISTS songs_by_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE IF EXISTS songs_by_session_and_user"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE IF EXISTS user_listen_song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [14]:
session.shutdown()
cluster.shutdown()